In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.3'
spark_version = 'spark-3.5.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,315 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,657 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,609 kB]
Get:13 http://archive.ubuntu.com/ubu

Exception: Unable to find py4j in /content/spark-3.5.3-bin-hadoop3/python, your SPARK_HOME may not be configured correctly

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

FileNotFoundError: [Errno 2] No such file or directory: '/content/spark-3.5.3-bin-hadoop3/./bin/spark-submit'

In [ ]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"


In [ ]:
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
SELECT YEAR(date_sold) AS year,
ROUND (AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY year
ORDER BY year"""
).show()

NameError: name 'spark' is not defined

In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
SELECT date_built,
ROUND(AVG(price), 2)AS average_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms == 3
GROUP BY date_built
ORDER BY date_built"""
).show()
ORDER BY year

In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql(""" SELECT date_built,
  ROUND(AVG(price), 2) AS average_price
  FROM home_sales
  WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >=2000
  GROUP BY date_built
  ORDER BY date_built"""

).show()
ORDER BY year

In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
spark.sql("""
SELECT ROUND (AVG(view), 2) AS average_view
FROM home_sales
WHERE price >= 350000""").show()


print("--- %s seconds ---" % (time.time() - start_time))
print(f"Execution Time: {time.time() - start_time} seconds")

--- 7.200241088867188e-05 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
spark.sql(""" SELECT ROUND (AVG(view), 2) AS average_view
  FROM home_sales
  WHERE price >=350000""").show()

print("--- %s seconds ---" % (time.time() - start_time))

print(f"Execution Time: {time.time() - start_time} seconds")

--- 5.459785461425781e-05 seconds ---


In [ ]:
if "date_built" in df.columns:
    # 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite".parquet("partitioned_home_sales"))

In [ ]:
# 11. Read the parquet formatted data.
p_df_p=spark.read.parquet("partitioned_home_sales")

In [ ]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView("p_home_sales_p")

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
spark.sql(""" SELECT ROUND(AVG(view), 2) AS average_view
  FROM home_sales
  WHERE price >=350000""").show()


print("--- %s seconds ---" % (time.time() - start_time))
print(f"Execution Time: {time.time() - start_time} seconds")

--- 7.104873657226562e-05 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')